# Documentation for Research Paper Chatbot 

### SE research paper chatbot

Group Name: [csusb_fall2024_cse6550_team4](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team4)

Instructor: Dr. Alzahrani, Nabeel

Course: CSE 6550: Software Engineer Concepts Fall 2024

Source: [Github](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team4)

# 1. Introduction

Purpose:

The purpose of this project is to create an AI-powered research paper chatbot that helps users extract, summarize, and understand content from academic papers. It will offer an interactive Q&A experience, providing accurate, contextually relevant answers to questions about specific sections, making complex research information more accessible and comprehensible.

Objective:

The Paper Chatbot enhances interaction with academic papers by allowing users to upload documents, ask questions, and receive summaries or clarifications. It simplifies extracting key information, aiding students, researchers, and professionals in efficiently understanding complex content.

Prerequisites:
Github, Docker, Jupyter Notebook, Python

# 2. Setup

Checking Python Version

- To ensure your system is compatible with the project, follow these steps:

- Run Command: Use ```!python --version``` to check the installed Python version.
- Verify Version: Confirm that the displayed version is 3.10 or higher.
- If the version is lower than 3.10, download and install the latest version from https://www.python.org/downloads/

Requirements:
- Python must be installed on your system.
- The project requires Python 3.10 or higher.

In [33]:
!python --version

Python 3.11.10


### Building Virtual Environment
- Install Tools: Installs the required Python packages (`ipykernel` and `virtualenv`) without showing any output or warnings during installation.
- Create Virtual Environment: Sets up a virtual environment named `chatbot`, a dedicated workspace for your project to manage its dependencies separately from the global system.

In [35]:
import os
import subprocess

# Suppress pip installation output
subprocess.run(
    "pip install ipykernel --root-user-action=ignore > NUL 2>&1", shell=True
)
subprocess.run(
    "pip install --user virtualenv --root-user-action=ignore --no-warn-script-location > NUL 2>&1",
    shell=True,
)

# Create the virtual environment
subprocess.run("python -m venv chatbot > NUL 2>&1", shell=True)

# Simulate activation (actual activation is done in the shell, this is just confirmation)
print("Virtual Environment Created")

Virtual Environment Created


### Installing dependencies
This command installs all the necessary Python packages for your project, including tools for data analysis, machine learning, AI, and visualization. It ensures your environment has everything required to build and run the project seamlessly.

In [39]:
pip install --root-user-action=ignore streamlit pandas jupyter langchain langchain-community langchain-huggingface langchain-text-splitters langchain-mistralai sentence-transformers faiss-cpu transformers matplotlib numpy plotly scikit-learn ipykernel roman yake pymilvus pydantic==2.5.2 mistralai

Note: you may need to restart the kernel to use updated packages.


### Initialization and Configuration of Constants

Purpose:
- Retrieve and Validate the `MISTRAL_API_KEY` Environment Variable

Input:
1. Environment variables:
`MISTRAL_API_KEY` (retrieved using `os.getenv()`).
2. Constants:
`MILVUS_URI`: Path to the Milvus vector database.
`MODEL_NAME`: Name of the embedding model.

Output:
- Prints confirmation messages:
    - Successful import of libraries and setup of constants.
    - Values of `MILVUS_URI` and `MODEL_NAME`.
- Raises an error if `MISTRAL_API_KEY` is not set:
- `ValueError: "MISTRAL_API_KEY environment variable not set."`

Processing:
- Checks for the `MISTRAL_API_KEY` in the environment:
     - If missing, raises a ValueError.
- Sets up constants (`MILVUS_URI` and `MODEL_NAME`) for subsequent operations.

In [36]:
# Validate MISTRAL_API_KEY and set constants
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")
if not MISTRAL_API_KEY:
    raise ValueError("MISTRAL_API_KEY environment variable not set.")
MILVUS_URI = "./milvus/milvus_vector.db"
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
print(f"MILVUS_URI: {MILVUS_URI}, MODEL_NAME: {MODEL_NAME}")

MILVUS_URI: ./milvus/milvus_vector.db, MODEL_NAME: sentence-transformers/all-MiniLM-L6-v2


### Configuring environment variables

Purpose: The code initializes an environment for NLP applications, specifically for document retrieval and embedding generation using machine learning models.

Input:

- Environment variables loaded from a `.env` file (`MISTRAL_API_KEY` and `USER_AGENT`)
- Configuration values such as `MILVUS_URI`, `MODEL_NAME`, and `CORPUS_SOURCE`

Output:
Confirms successful setup of the environment, document source readiness, and embedding model configuration for future processing.

Processing:
- Environment setup: Loads environment variables to set up API access and a user agent for HTTP requests.
- Configuration: Prepares paths and settings for embedding models and vector storage.
- Embedding preparation: Sets up a model (`sentence-transformers/all-MiniLM-L6-v2`) for processing documents and generating embeddings.
- Document retrieval: References a document source URL (`CORPUS_SOURCE`) as a placeholder for further processing or document loading.

In [3]:
from dotenv import load_dotenv
import os 

# Load environment variables from .env file
load_dotenv()  # This loads environment variables defined in a .env file into the program

# Retrieve the Mistral API key from the environment for authentication
MISTRAL_API_KEY = os.getenv("MISTRAL_API_KEY")  
# Set the USER_AGENT variable
USER_AGENT = os.getenv("USER_AGENT", "my_custom_user_agent")  
os.environ["USER_AGENT"] = USER_AGENT 

# Configuration settings for Milvus database, model name, and corpus source
MILVUS_URI = "./milvus/milvus_vector.db"  
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  
CORPUS_SOURCE = 'https://dl.acm.org/doi/proceedings/10.1145/3597503'  

# Print confirmation of setup
print("Environment initialized, documents loaded, embeddings configured.")  # Confirm successful setup

Environment initialized, documents loaded, embeddings configured.


# 3. Building the Chatbot

### Creating Hugging Face Embedding Function
Purpose: To create and return an embedding function configured with a specified NLP model for generating text embedding

Input:
- Model Name (`MODEL_NAME`): A pre-defined string representing the name of the model (e.g., `"sentence-transformers/all-MiniLM-L6-v2"`). This value is set globally and used to initialize the embedding function.

Output:
Returns an embedding function configured with the given model. This function can then be used to generate embeddings for text data.

Processing:
- The function creates an instance of `HuggingFaceEmbeddings` using `MODEL_NAME`, setting up an embedding generator based on the specified model.
- It returns this configured embedding generator for use in NLP tasks like document analysis or semantic search.
- It takes 2 minutes to run

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

#  Specify the HuggingFace model for embeddings
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  

def get_embedding_function():
    """
    Returns the embedding function for the specified model.

    Returns:
        HuggingFaceEmbeddings: An embedding function object.
    """
    # Initialize the HuggingFace embedding function with the specified model name
    embedding_function = HuggingFaceEmbeddings(model_name=MODEL_NAME)
    
    # Print a message confirming the embedding function has been created
    print(f"Embedding function created using model: {MODEL_NAME}")
    
    # Return the initialized embedding function
    return embedding_function

# Call the embeddin function and display the output
embedding_fn = get_embedding_function()  
print(f"Embedding function initialized: {embedding_fn}")  

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding function created using model: sentence-transformers/all-MiniLM-L6-v2
Embedding function initialized: client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='sentence-transformers/all-MiniLM-L6-v2' cache_folder=None model_kwargs={} encode_kwargs={} multi_process=False show_progress=False


### Prompt Generator

Purpose: To set up configurations for an NLP system and provide a placeholder function for generating prompts.

Input:
- Model Name: A string (`"sentence-transformers/all-MiniLM-L6-v2"`) representing the embedding model.
- Milvus URI: A string (`"./milvus/milvus_vector.db"`) specifying the location of the Milvus vector database.

Output:
- A configured environment ready for NLP tasks.
- `create_prompt()` function returns a string template to be used for generating detailed research summaries.

Processing:
- Configuration: Sets the model name for embedding generation and the URI for the vector database connection.
- Prompt creation: Defines a simple placeholder function create_prompt() that returns a formatted prompt template for use in text generation.
- - It takes 2 minutes to run

In [5]:
# Specifies the HuggingFace model for embedding generation and specifies the URI for the Milvus vector database
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  
MILVUS_URI = "./milvus/milvus_vector.db"  

# Placeholder function for creating a prompt
def create_prompt():
    """
    Returns a placeholder prompt template.

    Returns:
        str: The prompt template.
    """
    # Define a placeholder prompt for generating research summaries
    prompt = "Provide a detailed summary of the latest research on: {input}"
    print(f"Prompt created: {prompt}")  # Print confirmation of the created prompt
    return prompt  # Return the created prompt template

# Display the constants and call the function
print(f"MODEL_NAME: {MODEL_NAME}")  # Print the model name for reference
print(f"MILVUS_URI: {MILVUS_URI}")  # Print the Milvus URI for reference

prompt_template = create_prompt()  # Call the function to create a prompt template
print(f"Returned prompt: {prompt_template}")  # Print the returned prompt template

MODEL_NAME: sentence-transformers/all-MiniLM-L6-v2
MILVUS_URI: ./milvus/milvus_vector.db
Prompt created: Provide a detailed summary of the latest research on: {input}
Returned prompt: Provide a detailed summary of the latest research on: {input}


### Vector store loader

Purpose: To simulate the loading of a vector store from a specified URI and provide an interface for data retrieval.

Input:
URI: A string representing the location of the vector database (e.g., `MILVUS_URI`).

Output:
`VectorStore` object with an as_retriever method that simulates returning itself for further operations. This acts as a placeholder for actual vector store functionality.

Processing:
- The function defines a placeholder `VectorStore` class with an `as_retriever` method that returns the `VectorStore` instance itself, simulating a `vector store` capable of retrieval operations.
- The function returns an instance of `VectorStore` initialized when called.
- - It takes 2 minutes to run

In [6]:
# Function for loading the vector store
def load_existing_db(uri):
    """
    Simulates loading an existing vector store from a given URI.

    Args:
        uri (str): The URI of the vector store database.

    Returns:
        VectorStore: A mock vector store class instance.
    """
    # Define a mock VectorStore class with an as_retriever method
    class VectorStore:
        def as_retriever(self):
            # Simulate returning a retriever instance
            return self

    print(f"Loading vector store from URI: {uri}")  # Print confirmation of the URI being used
    return VectorStore()  # Return an instance of the mock VectorStore class

# Specify the URI for the vector store
MILVUS_URI = "./milvus/milvus_vector.db"  # Path to the Milvus vector database

# Call the function to load the vector store
vector_store = load_existing_db(MILVUS_URI)
print(f"Vector store loaded: {vector_store}")  # Print confirmation that the vector store was loaded successfully

Loading vector store from URI: ./milvus/milvus_vector.db
Vector store loaded: <__main__.load_existing_db.<locals>.VectorStore object at 0xffff3d91dfd0>


### Document Chain Placeholder

Purpose: To serve as a placeholder function for creating a document processing chain using a model and a prompt.

Input:
- Model: An object representing an NLP model that will be used in the document chain.
- Prompt: A string or object representing the prompt template to guide document processing.

Output:

Returns a string: "Document chain here." to signify that a document processing chain has been set up (used for demonstration or placeholder purposes).

Processing:
- The function takes the model and prompt as arguments but currently does not perform any operations on them.
- It returns a placeholder string indicating that a document chain has been created.
- - It takes 2 minutes to run

In [7]:
# Placeholder function for creating a document chain
def create_stuff_documents_chain(model, prompt):
    """
    Simulates the creation of a document chain.

    Args:
        model (str): The name of the model to use.
        prompt (str): The prompt template.

    Returns:
        str: A placeholder message indicating the document chain creation.
    """
    # Print the model being used for creating the document chain
    print(f"Creating document chain with model: {model}")
    
    # Print the prompt being used for the document chain
    print(f"Using prompt: {prompt}")
    
    # Return a placeholder message for the document chain
    return "Document chain here."


MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"  

# Define a test prompt for the document chain
test_prompt = "Provide a detailed summary of the latest research on: {input}"  

# Call the function to simulate creating a document chain
document_chain = create_stuff_documents_chain(MODEL_NAME, test_prompt)

# Print the result of the document chain creation
print(f"Returned document chain: {document_chain}")  

Creating document chain with model: sentence-transformers/all-MiniLM-L6-v2
Using prompt: Provide a detailed summary of the latest research on: {input}
Returned document chain: Document chain here.


# 4. Improving the Chatbot

### Improving Chatbot with RAG and Embeddings
Purpose: To create and manage a vector store using Milvus, which stores embeddings generated from web documents for NLP applications.

Input:
- Documents: Text content loaded from the web through the `load_documents_from_web()` function.
- Embedding Function: A function that converts document text into numerical embeddings.
- URI: A file path (`'./milvus/milvus_vector.db'`) specifying the location of the Milvus vector database.

Output:
- Prints console messages at each step, confirming the successful execution of loading, processing, and vector store management.
- If errors occur, an exception message is printed.

Processing:
- Document loading: Retrieves documents from a web source.
- Document splitting: Splits the loaded documents into manageable chunks.
- Embedding generation: Uses an embedding function to generate fixed-size numerical embeddings for each document chunk.
- Vector store creation:
   - Connects to Milvus.
   - Checks if a collection ("research_paper_chatbot") exists. If not, creates a new collection schema.
   - Inserts document embeddings into the collection.
- Vector store loading: Loads an existing vector store collection from Milvus for retrieval operations
- It takes 2 minutes to run

In [8]:
import os
from tqdm import tqdm
from pymilvus import (
    connections,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
    utility
)

# Suppress tqdm warnings by setting environment variable
os.environ['TQDM_DISABLE'] = '1'

# Placeholder function to simulate loading documents from the web
def load_documents_from_web():
    return ["Document 1 content", "Document 2 content", "Document 3 content"]

# Splits documents into smaller chunks
def split_documents(documents):
    return [doc for doc in documents]

# Placeholder function to simulate getting an embedding function
def get_embedding_function():
    def embedding_function(doc):
        return [0.1] * 512  # Example fixed-size embedding
    return embedding_function

def create_vector_store(docs, embeddings, uri):
    # Create the directory if it does not exist
    head = os.path.split(uri)
    os.makedirs(head[0], exist_ok=True)
    print("Directory created for vector store if it did not exist")

    # Connect to the Milvus database
    connections.connect("default", uri=uri)
    print("Connected to the Milvus database")

    # Define collection name
    collection_name = "research_paper_chatbot"

    # Check if the collection already exists
    if utility.has_collection(collection_name):
        print("Collection already exists. Loading existing Vector Store.")
        vector_store = Collection(name=collection_name)
        print("Existing Vector Store Loaded")
    else:
        print("Creating new Vector Store...")
        fields = [
            FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=512),
            FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        ]
        schema = CollectionSchema(fields=fields, description="Collection for research paper embeddings")
        collection = Collection(name=collection_name, schema=schema)
        print("New Vector Store Created with provided documents")

        # Insert documents into the vector store
        for doc in docs:
            embedding = embeddings(doc)
            collection.insert([[embedding]])

    return collection

def load_exisiting_db(uri):
    collection_name = "research_paper_chatbot"
    vector_store = Collection(name=collection_name)
    print("Loaded existing Vector Store from Milvus database")
    return vector_store

if __name__ == '__main__':
    try:
        print("Loading documents from the web...")
        documents = load_documents_from_web()
        print(f"Loaded {len(documents)} documents from the web.")

        print("Splitting documents into chunks...")
        docs = split_documents(documents)
        print(f"Split into {len(docs)} chunks.")

        print("Getting embedding function...")
        embeddings = get_embedding_function()

        uri = './milvus/milvus_vector.db'

        print("Creating vector store...")
        vector_store = create_vector_store(docs, embeddings, uri)
        print("Vector store created successfully.")

        print("Loading existing vector store...")
        loaded_vector_store = load_exisiting_db(uri)
        print("Loaded existing vector store successfully.")

    except Exception as e:
        print(f"An error occurred: {e}")

    print("Finished operations.")

Loading documents from the web...
Loaded 3 documents from the web.
Splitting documents into chunks...
Split into 3 chunks.
Getting embedding function...
Creating vector store...
Directory created for vector store if it did not exist
Connected to the Milvus database
Creating new Vector Store...
New Vector Store Created with provided documents
Vector store created successfully.
Loading existing vector store...
Loaded existing Vector Store from Milvus database
Loaded existing vector store successfully.
Finished operations.


### Query Response Generator
Purpose: To provide an entry point for querying a Retrieval-Augmented Generation (RAG) system, generate a response using NLP models, and include source references


Input:
Query: A string containing the user's question or request for information

Output:

Response: A string containing the generated answer with source references.
Sources: A list of URLs or source identifiers referenced in the response.

Processing:
1. Model and Prompt Initialization:
    - Loads a ChatMistralAI model and creates a prompt template with create_prompt() for structured input.
2. Vector Store Retrieval:
    - Loads an existing vector store from Milvus and creates a retriever to fetch relevant documents.
3. Document Chain Creation:
    - Uses `create_stuff_documents_chain()` to set up a document processing chain.
4. Retrieval Chain Setup:
    - Constructs a retrieval chain that integrates the retriever and document chain to process the query.
5. Query Handling and Response Generation:
    - Generates a response using the retrieval chain.
    - Catches and handles HTTPStatusError for handling service load issues (e.g., error 429 for high traffic).
6. Source Attribution:
    - Extracts and formats up to four unique sources from the response context to include in the output.
    - Appends source references to the generated response.
- It takes 2 minutes to run 

In [9]:
# Define the prompt template
from langchain_core.prompts import PromptTemplate

def create_prompt():
    """
    Creates a prompt template for the AI assistant.

    Returns:
        PromptTemplate: The created prompt template.
    """
    PROMPT_TEMPLATE = """
    Human: You are an AI assistant, and provides answers to questions by using fact-based and statistical information when possible.
    Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
    Only use the information provided in the <context> tags.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    <context>
    {context}
    </context>

    <question>
    {input}
    </question>

    The response should be specific and use statistics or numbers when possible.

    Assistant:"""

    # Create a PromptTemplate instance
    prompt = PromptTemplate(
        template=PROMPT_TEMPLATE, input_variables=["context", "question"]
    )
    print("Prompt Created Successfully!")
    print(f"Prompt Template:\n{PROMPT_TEMPLATE.strip()}")
    return prompt

# Call the function and print the returned value
created_prompt = create_prompt()
print(f"Returned PromptTemplate: {created_prompt}")

Prompt Created Successfully!
Prompt Template:
Human: You are an AI assistant, and provides answers to questions by using fact-based and statistical information when possible.
    Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.
    Only use the information provided in the <context> tags.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    <context>
    {context}
    </context>

    <question>
    {input}
    </question>

    The response should be specific and use statistics or numbers when possible.

    Assistant:
Returned PromptTemplate: input_variables=['context', 'input'] template="\n    Human: You are an AI assistant, and provides answers to questions by using fact-based and statistical information when possible.\n    Use the following pieces of information to provide a concise answer to the question enclosed in <question> tags.\n    Only use the information provided in

 ### Adding Evaluation Metrics with Confusion Matrix

Purpose: To manage and store performance metrics related to a chatbot's classification performance (e.g., True Positives, False Positives, Accuracy, etc.).

Inputs:
- `increment_value (integer)`: Value to increment or update specific metrics.
- `metric (string)`: The name of the metric to update.
- `columns (string)`: The columns to fetch from the database.

Processing:
- Interacts with an database to execute queries.
- Performs necessary calculations for metrics like sensitivity, specificity, precision, recall, F1 score. 
- Safely handles division to avoid division by zero errors.

Outputs:
Updates or retrieves performance metrics stored in the database.

- It takes 2 minutes to run

In [10]:
def update_performance_metrics(self):
    """
    Recalculate and update the performance metrics in the database.
    """
    # Fetch metrics from the database
    print("Fetching performance metrics from the database...")
    metrics = self.get_performance_metrics('true_positive, true_negative, false_positive, false_negative')
    print(f"Metrics retrieved: {metrics}")  # Print the metrics retrieved for debugging

    # Calculate accuracy
    accuracy = self.safe_division(
        metrics['true_positive'] + metrics['true_negative'], 
        metrics['true_positive'] + metrics['true_negative'] + metrics['false_positive'] + metrics['false_negative']
    )
    
    # Calculate precision
    precision = self.safe_division(
        metrics['true_positive'], 
        metrics['true_positive'] + metrics['false_positive']
    )
    
    # Calculate sensitivity (also known as recall)
    sensitivity = self.safe_division(
        metrics['true_positive'], 
        metrics['true_positive'] + metrics['false_negative']
    )
    
    # Calculate specificity
    specificity = self.safe_division(
        metrics['true_negative'], 
        metrics['true_negative'] + metrics['false_positive']
    )
    
    # Calculate recall
    recall = self.safe_division(
        metrics['true_positive'], 
        metrics['true_positive'] + metrics['false_negative']
    )

    # Print calculated metrics for debugging
    print(f"Calculated Metrics:\n"
          f"Accuracy: {accuracy}\n"
          f"Precision: {precision}\n"
          f"Sensitivity: {sensitivity}\n"
          f"Specificity: {specificity}\n"
          f"Recall: {recall}")

    # Calculate F1 score if precision and sensitivity are available
    if precision and sensitivity:
        f1_score = self.safe_division(2 * precision * sensitivity, precision + sensitivity)
    else:
        f1_score = None  # Handle cases where F1 score cannot be calculated

    print(f"F1 Score: {f1_score}")  # Print the F1 score for debugging

    # Update the metrics in the database
    print("Updating performance metrics in the database...")
    with self.connection:
        self.connection.execute('''
            UPDATE performance_metrics
            SET accuracy = ?, precision = ?, sensitivity = ?, specificity = ?, f1_score = ?, recall = ?
            WHERE id = 1
        ''', (accuracy, precision, sensitivity, specificity, f1_score, recall))
    print("Performance metrics updated successfully.")  # Confirm the update

### Retrieve an answer from RAG.

Preprocessing:

- The input query is processed using the `preprocess_query` function. This function cleans the query by:
- Removing non-alphanumeric characters (e.g., punctuation).
- Converting the query to lowercase for better matching during the search.

Input:

- The user enters a query in the text box `prompt_input`.

Output:

- The result is displayed in the output widget. This includes:
- The generated response.
- A list of research papers with titles and links.

Processing:

- The cleaned query is then passed to the `query_rag` function, which:
- Fetches research papers related to the query using the arXiv API.
- Simulates generating a response to the query (in a real-world setup, this could use an NLP model like GPT to generate more complex answers).
- Extracts paper titles and links from the API response.

In [31]:
import ipywidgets as widgets
from IPython.display import display
import requests
import re

# Create a text input widget for the user to enter a prompt
prompt_input = widgets.Text(
    value='', 
    placeholder='Enter your query here...',
    description='Prompt:',
    layout=widgets.Layout(width='80%')
)

# Create a button to submit the prompt
submit_button = widgets.Button(
    description='Submit',
    button_style='primary',
    tooltip='Click to submit your query'
)

# Create an output widget to display the response
output = widgets.Output()

# Helper function to search for relevant papers in arXiv
def search_arxiv(query):
    base_url = "http://export.arxiv.org/api/query?"
    params = {
        "search_query": f"all:{query}",
        "start": 0,
        "max_results": 5
    }
    response = requests.get(base_url, params=params)
    return response.text

# Preprocessing function: Clean the query for better search
def preprocess_query(query):
    # Remove non-alphanumeric characters and convert to lowercase
    query_cleaned = re.sub(r'[^\w\s]', '', query).strip().lower()
    return query_cleaned

# Processing function to generate answer and retrieve research papers dynamically
def query_rag(query):
    # Preprocess the input query
    query_cleaned = preprocess_query(query)
    
    # Querying arXiv for relevant research papers
    papers_xml = search_arxiv(query_cleaned)
    
    # Simulating response generation (use a proper NLP model for real use)
    response = f"Based on the query: '{query}', the system suggests the following research papers as references."
    
    # Parse arXiv results to extract paper titles and links
    papers = []
    # Find all the paper titles and links in the XML response
    paper_titles = re.findall(r"<title>(.*?)</title>", papers_xml)
    paper_links = re.findall(r"<id>(.*?)</id>", papers_xml)
    
    # Skip the first result (query metadata) and pair the titles with their links
    for title, link in zip(paper_titles[1:], paper_links[1:]):
        papers.append((title.strip(), link.strip()))
    
    return response, papers

# Function to handle button click and display the response
def on_submit_click(change):
    query = prompt_input.value
    with output:
        output.clear_output()  # Clear previous output
        
        # Process the query and retrieve the response
        response, papers = query_rag(query)
        
        # Display the processed response
        print(f"Query: {query}")
        print("\nResponse:")
        print(response)
        
        # Display the suggested research papers
        print("\nSuggested Papers:")
        if papers:
            for title, link in papers:
                print(f"Title: {title}\nLink: {link}\n")
        else:
            print("No research papers found for the given query.")

# Attach the button click event to the handler
submit_button.on_click(on_submit_click)

# 5. Testing the Chatbot

Purpose: Test the chatbot with a sample query

Input: Sample queries and documents. 

Output: Responses and any errors encountered. 

Processing: Test the RAG model’s response to ensure correctness.

- It takes 2 minutes to run

In [32]:
# Display the widgets
display(prompt_input, submit_button, output)

Text(value='', description='Prompt:', layout=Layout(width='80%'), placeholder='Enter your query here...')

Button(button_style='primary', description='Submit', style=ButtonStyle(), tooltip='Click to submit your query'…

Output()

# 6. Conclusion

Recap: We set up a retrieval-augmented generation (RAG) chatbot using LangChain and Milvus, integrated PDF loading, and improved its functionality.

Next Steps: Consider enhancing the chatbot with real-time data retrieval or using more advanced NLP techniques.

Resources: For more details, visit [Github](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team4) and 
[Wiki for reference](https://github.com/DrAlzahraniProjects/csusb_fall2024_cse6550_team4/wiki)